In [3]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/')
import os, findspark
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
import zipfile
import zipimport
import io
import logging
logging.getLogger().setLevel(logging.INFO)
from pyspark.sql import functions
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc
import zlib
import importlib
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call
import pubmed_parser as pp
import pyarrow as pa
import subprocess

#sys.path.append('/home/ananth/nsf_data_ingestion/')

from nsf_data_ingestion.config import spark_config
from nsf_data_ingestion.objects import data_source_params

namenode_url = "hdfs://ist-deacuna-n1.syr.edu:8020/"

In [4]:
def create_session():
    logging.info('Creating Spark Session....')
    spark = SparkSession.builder.config("spark.executor.instances", spark_config.exec_instance).\
                                 config("spark.executor.memory", spark_config.exec_mem).\
                                 config('spark.executor.cores', spark_config.exec_cores).\
                                 config('spark.cores.max', spark_config.exec_max_cores).\
                                 config('spark.jars', '/home/eileen/nsf_data_ingestion/libraries/spark-xml_2.11-0.5.0.jar').\
                                 appName('medline_parquet_test').getOrCreate()
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/pubmed_parser-0.1.0-py3.6.egg')
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/Unidecode-1.1.1-py3.6.egg')
#         logging.info('Adding Libraries' + str(library))
#         spark.sparkContext.addPyFile(library)    # adding libraries
#         spark.sparkContext.addPyFile(library)

        
    return spark
spark = create_session()


INFO:root:Creating Spark Session....


In [18]:
hdfs = pa.hdfs.connect()
file_paths = hdfs.ls('test/test')
hdfs.ls('test/test')

['/user/eileen/test/test/pubmed21n0001.xml.gz',
 '/user/eileen/test/test/pubmed21n0002.xml.gz',
 '/user/eileen/test/test/pubmed21n0003.xml.gz',
 '/user/eileen/test/test/pubmed21n0004.xml.gz',
 '/user/eileen/test/test/pubmed21n0005.xml.gz',
 '/user/eileen/test/test/pubmed21n0006.xml.gz',
 '/user/eileen/test/test/pubmed21n0007.xml.gz',
 '/user/eileen/test/test/pubmed21n0008.xml.gz',
 '/user/eileen/test/test/pubmed21n0009.xml.gz']

In [7]:
sc = spark.sparkContext

In [25]:
rdd0 = sc.parallelize(list(range(10000))).repartition(1000)

In [39]:
def get_hostnode(e):
    import sys
    import socket
    
    return str(sys.version_info), str(socket.gethostname())


In [40]:
def check_version(e):
    import pyarrow as pa
    hdfs = pa.hdfs.connect()
    
    return str(pa.__version__), str(hdfs)

In [41]:
rdd0.map(check_version).distinct().collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 20.0 failed 4 times, most recent failure: Lost task 2.3 in stage 20.0 (TID 6578, ist-deacuna-n5.syr.edu, executor 53): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/python/pyspark/rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/cloudera/parcels/SPARK2-2.4.0.c

In [34]:
hdfs = pa.hdfs.connect()

In [36]:
print(hdfs)

In [27]:
rdd0.map(get_hostnode).distinct().collect()

[("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
  'ist-deacuna-n6'),
 ("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
  'ist-deacuna-n7'),
 ("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
  'ist-deacuna-n4'),
 ("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
  'ist-deacuna-n3'),
 ("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
  'ist-deacuna-n2'),
 ("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
  'ist-deacuna-n5'),
 ("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
  'ist-deacuna-n9'),
 ("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
  'ist-deacuna-n8')]

In [28]:
get_hostnode(2)

("sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)",
 'ist-deacuna-n1')

In [31]:
!/home/tozeng/anaconda3/bin/python --version

Python 3.7.9


In [ ]:
test_p = spark.sparkContext.parallelize(file_paths)
test_p.collect()

In [9]:
def test_medline_parquet(gzip):
    import pubmed_parser as pa
    dicts = pa.parse_medline_xml(gzip[1])
    return [Row(**doc) for doc in dicts]

In [10]:
docs = spark.sparkContext.wholeTextFiles('test/test/pubmed21n0001.xml.gz')
dicts = docs.flatMap(test_medline_parquet)
dicts.toDF()

DataFrame[abstract: string, affiliation: string, author: string, country: string, delete: boolean, issn_linking: string, journal: string, keywords: string, medline_ta: string, mesh_terms: string, nlm_unique_id: string, other_id: string, pmc: string, pmid: string, pubdate: string, title: string]

In [11]:

def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    
    filepath = gzip_str[0]
    xml_string = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]



In [14]:
docs = spark.sparkContext.wholeTextFiles('test/test/pubmed21n0001.xml.gz')
dicts = docs.flatMap(parse_gzip_medline_str)
dicts.take(10)



[Row(abstract='', affiliation='', author='AB Makar; KE McMartin; M Palese; TR Tephly', country='United States', delete=False, file_name='pubmed21n0001.xml.gz', issn_linking='0006-2944', journal='', keywords='', medline_ta='Biochem Med', mesh_terms='D000445:Aldehyde Oxidoreductases; D000818:Animals; D001826:Body Fluids; D002245:Carbon Dioxide; D005561:Formates; D000882:Haplorhini; D006801:Humans; D006863:Hydrogen-Ion Concentration; D007700:Kinetics; D000432:Methanol; D008722:Methods; D011549:Pseudomonas', nlm_unique_id='0151424', other_id='', pmc='', pmid='1', pubdate='1975', title='Formate assay in body fluids: application in methanol poisoning.'),
 Row(abstract='', affiliation='', author='KS Bose; RH Sarma', country='United States', delete=False, file_name='pubmed21n0001.xml.gz', issn_linking='0006-291X', journal='', keywords='', medline_ta='Biochem Biophys Res Commun', mesh_terms='D005583:Fourier Analysis; D009682:Magnetic Resonance Spectroscopy; D008958:Models, Molecular; D008968:Mo

In [ ]:
!hdfs dfs 

In [ ]:
spark.stop()

In [ ]:
def test_medline_parquet(file_path):
    import subprocess
    (ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'test/test/'])
    
    return (out,err)

In [ ]:
def run_cmd(args_list):
        """
        run linux commands
        """
        # import subprocess
        print('Running system command: {0}'.format(' '.join(args_list)))
        proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s_output, s_err = proc.communicate()
        s_return =  proc.returncode
        return s_return, s_output, s_err 

In [ ]:
(ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'test/test/'])
out.decode('ascii').split('\n')

In [ ]:
err = test_p.map(test_medline_parquet).take(1)

In [ ]:
err[0][1].decode('ascii').split('\n')

In [ ]:
tet = spark.sparkContext.wholeTextFiles('test/test/*.xml.gz')

In [ ]:
tet.map(lambda x : x[0]).take(1)

In [ ]:
pa.hdfs.connect()

In [ ]:
spark.stop()